This tutorial and the assets can be downloaded as part of the [Wallaroo Tutorials repository](https://github.com/WallarooLabs/Wallaroo_Tutorials/blob/wallaroo2024.4_tutorials/wallaroo-model-operations-tutorials/inference/parallel-infer-tutorial).

## Wallaroo SDK Parallel Inference Tutorial

This tutorial will focus on the Pipeline method `parallel_infer`, which allows a List of data to be submitted to a Wallaroo instance for parallel inference requests.  This provides high speed increases in situations where data has to be broken up for size and memory needs, data is requested from multiple sources and submitted in a single request, or other use cases.

For this example we will be using an open source model that uses an [Aloha CNN LSTM model](https://www.researchgate.net/publication/348920204_Using_Auxiliary_Inputs_in_Deep_Learning_Models_for_Detecting_DGA-based_Domain_Names) for classifying Domain names as being either legitimate or being used for nefarious purposes such as malware distribution.

This example is a "toy" example.  A proper use case would be in situations where the input data is multiple gigabytes in size, or where the model can only operate in batch mode.

## Tutorial Goals

* Create a workspace for our work.
* Upload the Aloha TensorFlow model.
* Create a pipeline that can ingest our submitted data, submit it to the model, and export the results.
* Run a sample inference through our pipeline by loading a file.
* Run a batch inference to show submitting a set of data to an inference request.
* Perform parallel inference via two methods:
  * Split a DataFrame into a List:
    * Split a DataFrame into a List of 1,000 separate DataFrames to simulate separate inference requests.
    * Submit the List of DataFrames sequentially and display how long this takes.
  * Submit the same List of DataFrames with `parallel_infer` and compare how long it takes compared to the sequential list method.
  * Submit a DataFrame directly into the `parallel_infer` method and compare how long it takes compared to the sequential list method.

## Prerequisites

* Wallaroo Version 2024.1 and above instance.

## Reference

[Wallaroo SDK Essentials Guide: Inference Management](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-inferences/#parallel-inferences)

## Steps

### Import Libraries

The first step will be to import our libraries.

In [1]:
import wallaroo
import asyncio 
import datetime

# used to display dataframe information without truncating
import pandas as pd
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)

# to display dataframe tables
from IPython.display import display

import warnings
warnings.filterwarnings('ignore')

### Open a Connection to Wallaroo

Next we connect to a Wallaroo instance through the Wallaroo client.  The Python library is included in the Wallaroo install and available through the Jupyter Hub interface provided with your Wallaroo environment.

This is accomplished using the `wallaroo.Client()` command, which provides a URL to grant the SDK permission to your specific Wallaroo environment.  When displayed, enter the URL into a browser and confirm permissions.  Store the connection into a variable that can be referenced later.

In [2]:
wl = wallaroo.Client()

### Create Workspace

We will create a workspace to manage our pipeline and models.  The following variables will set the name of our sample workspace then set it as the current workspace.

Workspace names are unique across the Wallaroo instance.  Verify either that another workspace with the same name does not exist, or that access is granted to the user.

In [3]:
workspace_name = f"alohaworkspace-parallel-infer"
pipeline_name = f"alohapipeline"
model_name = f"alohamodel"
model_file_name = './models/alohacnnlstm.zip'

In [4]:
workspace = wl.get_workspace(name=workspace_name, create_if_not_exist=True)

wl.set_current_workspace(workspace)

aloha_pipeline = wl.build_pipeline(pipeline_name)
aloha_pipeline

name,alohapipeline
created,2024-07-25 19:41:18.960652+00:00
last_updated,2024-07-25 19:41:18.960652+00:00
deployed,(none)
workspace_id,28
workspace_name,alohaworkspace-parallel-infer
arch,None
accel,None
tags,
versions,2d894b43-6d1b-4965-aff6-eb8607b65d08
steps,


### Upload the Models

Now we will upload our models.  Note that for this example we are applying the model from a .ZIP file.  The Aloha model is a [protobuf](https://developers.google.com/protocol-buffers) file that has been defined for evaluating web pages, and we will configure it to use data in the `tensorflow` format.

In [5]:
model = (wl.upload_model(model_name, 
                        model_file_name, 
                        framework=wallaroo.framework.Framework.TENSORFLOW)
        )

### Deploy a model

Now that we have a model that we want to use we will create a deployment for it. 

We will tell the deployment we are using a tensorflow model and give the deployment name and the configuration we want for the deployment.

In [6]:
aloha_pipeline.add_model_step(model)

name,alohapipeline
created,2024-07-25 19:41:18.960652+00:00
last_updated,2024-07-25 19:41:18.960652+00:00
deployed,(none)
workspace_id,28
workspace_name,alohaworkspace-parallel-infer
arch,None
accel,None
tags,
versions,2d894b43-6d1b-4965-aff6-eb8607b65d08
steps,


In [7]:
REPLICAS = 2
deployment_config = (wallaroo.DeploymentConfigBuilder()
    .replica_count(REPLICAS)
    .build())

In [ ]:
aloha_pipeline.deploy(deployment_config =deployment_config)

We can verify that the pipeline is running and list what models are associated with it.

In [10]:
aloha_pipeline.status()

{'status': 'Running',
 'details': [],
 'engines': [{'ip': '10.28.1.2',
   'name': 'engine-f9bd787d9-zz55j',
   'status': 'Running',
   'reason': None,
   'details': [],
   'pipeline_statuses': {'pipelines': [{'id': 'alohapipeline',
      'status': 'Running',
      'version': '0904f92a-1401-45f9-a6ef-aac8402daff5'}]},
   'model_statuses': {'models': [{'name': 'alohamodel',
      'sha': 'd71d9ffc61aaac58c2b1ed70a2db13d1416fb9d3f5b891e5e4e2e97180fe22f8',
      'status': 'Running',
      'version': 'fb36dcf3-9827-42e9-89c0-d607c668b2d6'}]}},
  {'ip': '10.28.5.5',
   'name': 'engine-f9bd787d9-d6xrg',
   'status': 'Running',
   'reason': None,
   'details': [],
   'pipeline_statuses': {'pipelines': [{'id': 'alohapipeline',
      'status': 'Running',
      'version': '0904f92a-1401-45f9-a6ef-aac8402daff5'}]},
   'model_statuses': {'models': [{'name': 'alohamodel',
      'sha': 'd71d9ffc61aaac58c2b1ed70a2db13d1416fb9d3f5b891e5e4e2e97180fe22f8',
      'status': 'Running',
      'version': 'fb36

### Sample Inferences

#### Infer 1 row

Now that the pipeline is deployed and our Aloha model is in place, we'll perform a smoke test to verify the pipeline is up and running properly.  We'll use the `infer_from_file` command to load a single encoded URL into the inference engine and print the results back out.

The result should tell us that the tokenized URL is legitimate (0) or fraud (1) based on the `out.main` field.  This sample data should return close to 0.

In [11]:
result = aloha_pipeline.infer_from_file('./data/data_1.df.json')

display(result)

,time,in.text_input,out.banjori,out.corebot,out.cryptolocker,out.dircrypt,out.gozi,out.kraken,out.locky,out.main,out.matsnu,out.pykspa,out.qakbot,out.ramdo,out.ramnit,out.simda,out.suppobox,anomaly.count
0,2024-07-25 19:42:43.272,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 28, 16, 32, 23, 29, 32, 30, 19, 26, 17]",[0.0015195842],[0.98291475],[0.012099553],[4.7591206e-05],[2.0289332e-05],[0.00031977228],[0.011029261],[0.997564],[0.010341614],[0.008038961],[0.016155045],[0.0062362333],[0.0009985747],[1.7933435e-26],[1.3889844e-27],0


## Parallel Inference via the Wallaroo SDK

### Parallel Inference Inference Request

The [await `wallaroo.pipeline.Pipeline.parallel_infer(tensor, timeout, num_parallel, retries)`](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-reference-guide/pipeline/#Pipeline.parallel_infer) **asynchronous** method performs an inference as defined by the pipeline steps.

#### Parallel Inference Inference Request Parameters

| Parameter | Type | Description |
|---|---|---|
| **tensor** | *pandas.DataFrame* OR *pyarrow.Table* (*Required*) | The data submitted to the pipeline for inference a [pandas.DataFrame](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html) or [Apache Arrow pyarrow.Table](https://arrow.apache.org/) |
| **timeout** | *Integer* (*Optional*) |A timeout in seconds before the inference throws an exception.  The default is 15 second per call to accommodate large, complex models.  Note that for a batch inference, this is **per list item** - with 10 inference requests, each would have a default timeout of 15 seconds. |
| **num_parallel** | *Integer* (*Optional*)  | The number of parallel threads used for the submission.  **This should be no more than four times the number of pipeline replicas**. |
| **retries** | *Integer* (*Optional*) | The number of retries per inference request submitted. |
| **dataset** | *List(String)* (*Optional*) | The dataset of the inference result to return.  By default this is set to ["*"] which returns ["time", "in", "out", "anomaly"]. Other available options - ["metadata"]. |
| **dataset_exclude** | *List(String)* (*Optional*) | The datasets to exclude in the inference results returned values. |

#### Parallel Inference Inference Request Returns

`wallaroo.pipeline.Pipeline.parallel_infer` returns one of the following based on the `tensor` **input** parameter.

* If a pandas DataFrame was submitted, a pandas DataFrame is returned.
* If an Apache Arrow table was submitted, an Apache Arrow table is returned.

The following fields are returned based on the `dataset` and `dataset_exclude` parameters.

| Parameter | Type | Description |
|---|---|---|
| **time** | *DateTime* | The DateTime of the returned inference result. |
| **in** | *Any* | The input parameters.  Each `in` dataset field correlates to the input field from the inference request.  For example, the inputs `['year_built', 'last_renovated', 'number_of_bedrooms']` generates the `in` dataset `['in.year_built', 'in.last_renovated', 'in.number_of_bedrooms'] |
| **out** | *Any* | The output parameters.  Each `out` dataset field correlates to the final output of the pipeline.  For example, if the final output is `['house_price', 'recommended_starting_bid']`, the output fields would be `['out.house_price', 'out.recommended_starting_bid]`.
| **anomaly** | *Any* | The detected anomalies based on the validations added to the pipeline.  The field `anomaly.count` is always included, which displays a count of all validations that returned `True`, which indicates a detected anomaly.  Other anomalies are listed by `anomaly.{validation_name}`.  For more information, see [Wallaroo Anomaly Detection](https://docs.wallaroo.ai/wallaroo-operations-guide/wallaroo-pipeline-management/wallaroo-model-anomaly-detection/).
| **meta** | `Dict` | Includes the following fields with the `metadata` dataset.  <ol><li>`metadata.elapsed`:A List of time in nanoseconds for:<ol><li>The time to serialize the input.</li><li>How long each step took.</li></ol></li><li>`metadata.last_model`: A dict with each Python step as:<ol><li>`model_name`: The name of the model in the pipeline step.</li><li>`model_sha` : The sha hash of the model in the pipeline step.</li></ol></li><li>`metadata.pipeline_version`: The pipeline version as a UUID value.</li><li>`metadata.partition`: The partition used to store the inference results from this pipeline.  This is mainly used when adding Wallaroo Server edge deployments to a published pipeline and separating the inference results from those edge deployments.</li></ol> |

#### Parallel Inference Examples

##### Sequential Inference Example

The first example will show taking a pandas DataFrame with 25,000 rows, splitting those into 25,000 separate DataFrames with one row each, then submitting them sequentially.  This simulates a situation where the data input sizes are so large they must be broken up for more efficient transmission and inferencing.

In [12]:
test_data = pd.read_json("./data/data_25k.df.json")
test_list = []

for index, row in test_data.head(1000).iterrows():
    test_list.append(row.to_frame('text_input').reset_index(drop=True))

In [13]:
#show the first row as an example
test_list[0]

,text_input
0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 28, 16, 32, 23, 29, 32, 30, 19, 26, 17]"


In [15]:
#
# Run the inference sequentially to establish a baseline
#
now = datetime.datetime.now()

results = []
for df in test_list:
    results.append(aloha_pipeline.infer(tensor=df, timeout=10))

total_sequential = datetime.datetime.now() - now

print(f"Elapsed = {total_sequential.total_seconds()} : {len(results)}")

Elapsed = 240.318378 : 1000


##### Parallel Infer with DataFrame Example

The following example shows using `wallaroo.pipeline.Pipeline.parallel_infer` with a **pandas DataFrames**.  It is automatically split for parallel inferences by the Wallaroo SDK.

We then compare it to the List of pandas DataFrames submitted sequentially.

In [17]:
timeout_secs=30
now = datetime.datetime.now()
parallel_results = await aloha_pipeline.parallel_infer(tensor=test_data.head(1000), 
                                                       timeout=timeout_secs, 
                                                       num_parallel=2*REPLICAS, 
                                                       retries=3)

total_parallel = datetime.datetime.now() - now
print(f"Elapsed_in_parallel = {total_parallel.total_seconds()} : {len(parallel_results)}")

Elapsed_in_parallel = 15.697349 : 1000


In [18]:
parallel_results.head(20)

,time,in.text_input,out.banjori,out.corebot,out.cryptolocker,out.dircrypt,out.gozi,out.kraken,out.locky,out.main,out.matsnu,out.pykspa,out.qakbot,out.ramdo,out.ramnit,out.simda,out.suppobox,anomaly.count
0,2024-07-25 19:48:11.910,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 28, 16, 32, 23, 29, 32, 30, 19, 26, 17]",[0.0015195842],[0.98291475],[0.012099553],[4.7591206e-05],[2.0289332e-05],[0.00031977228],[0.011029261],[0.997564],[0.010341614],[0.008038961],[0.016155045],[0.0062362333],[0.0009985747],[1.7933435e-26],[1.3889844e-27],0
1,2024-07-25 19:48:11.910,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 30, 20, 19, 27, 14, 17, 24, 13, 23, 20, 18, 35, 18, 22, 23]",[7.447168e-18],[6.735899e-08],[0.17081985],[1.3220147e-09],[1.2758705e-24],[0.22559549],[0.3420985],[0.99999994],[0.3080186],[0.1828217],[3.8022332e-11],[0.2062254],[0.15215823],[1.1701893e-30],[3.1513734e-38],0
2,2024-07-25 19:48:11.898,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 25, 33, 25, 36, 25, 31, 14, 32, 36, 25, 12, 35, 34, 30, 28, 27, 24, 29, 27]",[2.8598976e-21],[9.301987e-08],[0.04445297],[6.163758e-09],[8.3497386e-23],[0.48234487],[0.263329],[1.0],[0.29800338],[0.22361773],[1.5238979e-06],[0.32820383],[0.029332481],[1.1995622e-31],[0.0],0
3,2024-07-25 19:48:11.909,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 17, 23, 22, 15, 12, 35, 34, 36, 12, 18, 24, 34, 32, 36, 12, 14, 16, 27, 22, 23]",[2.1387213e-15],[3.8817338e-10],[0.04559972],[1.9090405e-07],[1.3140474e-25],[0.5954264],[0.17374131],[0.9999997],[0.23151566],[0.1759168],[1.0876193e-09],[0.21832275],[0.012869264],[6.1588803e-28],[1.4386127e-35],0
4,2024-07-25 19:48:11.993,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 32, 13, 14, 12, 33, 16, 23, 15, 22, 30, 28, 26, 12, 16, 32, 37, 29, 22, 28, 22, 16, 27, 32]",[9.4533425e-15],[7.091165e-10],[0.04981512],[5.2914135e-09],[7.413152e-19],[1.5504575e-13],[1.0791892e-15],[0.9999989],[1.5003076e-15],[0.3307571],[2.625885e-07],[0.50362796],[0.020393759],[0.0],[2.329197e-38],0
5,2024-07-25 19:48:11.998,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 17, 29, 20, 33, 13, 36, 35, 30, 21, 29, 17, 26, 19, 25, 36, 14, 23, 16, 18, 15, 21, 18, 28, 35, 19]",[1.7247285e-17],[8.1354194e-08],[0.013697129],[5.6086392e-11],[1.4032912e-17],[0.4946911],[0.11978862],[0.99999994],[0.19000016],[0.10596694],[5.524429e-06],[0.24210057],[0.0069435085],[1.2804814e-34],[9.482465e-35],0
6,2024-07-25 19:48:11.997,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 22, 36, 14, 12, 23, 14, 13, 20, 20, 23, 27, 36, 29, 35, 19, 33, 22, 25, 26, 32, 21]",[5.5500585e-18],[3.3608643e-07],[0.023452938],[1.1318812e-10],[1.0496957e-22],[0.23692927],[0.064456925],[0.99999183],[0.07306592],[0.06499427],[1.4302767e-08],[0.11925243],[0.0011031023],[1.5206224e-32],[0.0],0
7,2024-07-25 19:48:11.998,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 36, 22, 28, 23, 20, 25, 21, 20, 16, 12, 33, 21, 14, 34, 34, 32, 19, 36, 17, 29, 26, 14, 29]",[3.9222568e-18],[1.407435e-10],[0.010946895],[8.202812e-11],[2.454965e-24],[0.42107278],[0.071240015],[0.9982491],[0.118182994],[0.08340969],[1.9207886e-09],[0.16958171],[0.0005199058],[0.0],[0.0],0
8,2024-07-25 19:48:12.049,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 27, 30, 33, 29, 37, 24, 33, 16, 20, 24]",[4.0574426e-11],[1.087887e-09],[0.17916852],[1.7313088e-06],[8.697261e-18],[9.197122e-16],[3.8521073e-17],[0.9999977],[3.2654394e-17],[0.32568428],[6.834302e-09],[0.37007827],[0.44918332],[0.0],[2.082403e-26],0
9,2024-07-25 19:48:12.054,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

## Undeploy Pipeline

When finished with our tests, we will undeploy the pipeline so we have the Kubernetes resources back for other tasks.  Note that if the deployment variable is unchanged aloha_pipeline.deploy() will restart the inference engine in the same configuration as before.

In [20]:
aloha_pipeline.undeploy()

name,alohapipeline
created,2024-07-25 19:41:18.960652+00:00
last_updated,2024-07-25 19:41:21.484892+00:00
deployed,False
workspace_id,28
workspace_name,alohaworkspace-parallel-infer
arch,x86
accel,none
tags,
versions,"0904f92a-1401-45f9-a6ef-aac8402daff5, 2d894b43-6d1b-4965-aff6-eb8607b65d08"
steps,alohamodel
